This notebook trains and tests the Sattelite Image with help of 5 Deep Learning Models

In [ ]:
import os
import numpy as np
import rasterio
from rasterio.plot import reshape_as_image
from sklearn.cluster import KMeans
import cv2
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from google.colab import drive
drive.mount('/content/drive')

Training

In [ ]:
import os, numpy as np, tensorflow as tf, rasterio
from tensorflow.keras import layers, models, optimizers
from sklearn.model_selection import train_test_split
from rasterio.windows import Window
import matplotlib.pyplot as plt
from tqdm import tqdm
RAW_IMAGE = "../data//raw/20241110_053942_45_24f7_3B_AnalyticMS_SR_8b_clip.tif"
RAW_MASK  = "../data/raw/20241110_053942_45_24f7_3B_AnalyticMS_SR_8b_clip_Hybrid_mask.tif"
OUT_DIR   = "../experiments_all"
os.makedirs(OUT_DIR, exist_ok=True)
IMG_SIZE = (256, 256)
BATCH_SIZE = 16
EPOCHS = 150
LR = 5e-4
def tile_raster_pair(image_path, mask_path, tile_size=IMG_SIZE, stride=None):
    if stride is None:
        stride = tile_size[0]
    imgs, msks = [], []
    with rasterio.open(image_path) as img_src, rasterio.open(mask_path) as mask_src:
        for top in range(0, img_src.height - tile_size[0] + 1, stride):
            for left in range(0, img_src.width - tile_size[1] + 1, stride):
                window = Window(left, top, tile_size[1], tile_size[0])
                img = np.moveaxis(img_src.read(window=window), 0, 2).astype(np.float32)
                mask = mask_src.read(1, window=window).astype(np.uint8)
                img = img / (np.max(img) + 1e-8)
                imgs.append(img)
                msks.append(np.expand_dims(mask, -1))
    return np.array(imgs), np.array(msks)

X, Y = tile_raster_pair(RAW_IMAGE, RAW_MASK)
print(f" Tiled: {len(X)} tiles — shape {X.shape}")
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
def conv_block(x, filters):
    x = layers.Conv2D(filters, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    x = layers.Conv2D(filters, 3, padding='same', activation='relu')(x)
    x = layers.BatchNormalization()(x)
    return x
def encoder_block(x, filters):
    c = conv_block(x, filters)
    p = layers.MaxPooling2D((2, 2))(c)
    return c, p
def decoder_block(x, skip, filters):
    x = layers.Conv2DTranspose(filters, (2, 2), strides=2, padding='same')(x)
    x = layers.concatenate([x, skip])
    x = conv_block(x, filters)
    return X
def build_unet(input_shape):
    inputs = layers.Input(shape=input_shape)
    c1, p1 = encoder_block(inputs, 64)
    c2, p2 = encoder_block(p1, 128)
    c3, p3 = encoder_block(p2, 256)
    c4, p4 = encoder_block(p3, 512)
    b = conv_block(p4, 1024)
    d1 = decoder_block(b, c4, 512)
    d2 = decoder_block(d1, c3, 256)
    d3 = decoder_block(d2, c2, 128)
    d4 = decoder_block(d3, c1, 64)
    outputs = layers.Conv2D(1, (1, 1), activation='sigmoid')(d4)
    return models.Model(inputs, outputs, name='U-Net')
def residual_block(x, filters):
    shortcut = layers.Conv2D(filters, (1, 1), padding="same")(x)
    x = layers.Conv2D(filters, (3, 3), padding="same", activation="relu")(x)
    x = layers.Conv2D(filters, (3, 3), padding="same")(x)
    x = layers.Add()([x, shortcut])
    return layers.Activation("relu")(x)

def build_resunet(input_shape):
    inputs = layers.Input(shape=input_shape)
    c1 = residual_block(inputs, 64); p1 = layers.MaxPooling2D(2)(c1)
    c2 = residual_block(p1, 128); p2 = layers.MaxPooling2D(2)(c2)
    c3 = residual_block(p2, 256); p3 = layers.MaxPooling2D(2)(c3)
    c4 = residual_block(p3, 512); p4 = layers.MaxPooling2D(2)(c4)
    b = residual_block(p4, 1024)
    u1 = layers.Conv2DTranspose(512, 2, strides=2, padding="same")(b)
    u1 = layers.Concatenate()([u1, c4]); d1 = residual_block(u1, 512)
    u2 = layers.Conv2DTranspose(256, 2, strides=2, padding="same")(d1)
    u2 = layers.Concatenate()([u2, c3]); d2 = residual_block(u2, 256)
    u3 = layers.Conv2DTranspose(128, 2, strides=2, padding="same")(d2)
    u3 = layers.Concatenate()([u3, c2]); d3 = residual_block(u3, 128)
    u4 = layers.Conv2DTranspose(64, 2, strides=2, padding="same")(d3)
    u4 = layers.Concatenate()([u4, c1]); d4 = residual_block(u4, 64)
    outputs = layers.Conv2D(1, 1, activation="sigmoid")(d4)
    return models.Model(inputs, outputs, name="ResU-Net")
def attention_gate(x, g, inter_channels):
    theta_x = layers.Conv2D(inter_channels, 2, strides=2, padding="same")(x)
    phi_g = layers.Conv2D(inter_channels, 1, padding="same")(g)
    add = layers.Add()([theta_x, phi_g])
    act = layers.Activation("relu")(add)
    psi = layers.Conv2D(1, 1, activation="sigmoid")(act)
    psi_up = layers.UpSampling2D(size=(2,2), interpolation="bilinear")(psi)
    return layers.Multiply()([x, psi_up])
def build_attnunet(input_shape):
    inputs = layers.Input(shape=input_shape)
    c1, p1 = encoder_block(inputs, 64)
    c2, p2 = encoder_block(p1, 128)
    c3, p3 = encoder_block(p2, 256)
    c4, p4 = encoder_block(p3, 512)
    b = conv_block(p4, 1024)
    g1 = layers.Conv2D(512, 1, padding="same")(b)
    att1 = attention_gate(c4, g1, 256)
    u1 = decoder_block(b, att1, 512)
    g2 = layers.Conv2D(256, 1, padding="same")(u1)
    att2 = attention_gate(c3, g2, 128)
    u2 = decoder_block(u1, att2, 256)
    g3 = layers.Conv2D(128, 1, padding="same")(u2)
    att3 = attention_gate(c2, g3, 64)
    u3 = decoder_block(u2, att3, 128)
    u4 = decoder_block(u3, c1, 64)
    outputs = layers.Conv2D(1, 1, activation="sigmoid")(u4)
    return models.Model(inputs, outputs, name="Attn-U-Net")
def build_attnresunet(input_shape):
    inputs = layers.Input(shape=input_shape)
    c1 = residual_block(inputs, 64); p1 = layers.MaxPooling2D(2)(c1)
    c2 = residual_block(p1, 128); p2 = layers.MaxPooling2D(2)(c2)
    c3 = residual_block(p2, 256); p3 = layers.MaxPooling2D(2)(c3)
    c4 = residual_block(p3, 512); p4 = layers.MaxPooling2D(2)(c4)
    b = residual_block(p4, 1024)
    g1 = layers.Conv2D(512, 1, padding="same")(b)
    att1 = attention_gate(c4, g1, 256)
    u1 = layers.Conv2DTranspose(512, 2, strides=2, padding="same")(b)
    u1 = layers.Concatenate()([u1, att1]); d1 = residual_block(u1, 512)
    g2 = layers.Conv2D(256, 1, padding="same")(d1)
    att2 = attention_gate(c3, g2, 128)
    u2 = layers.Conv2DTranspose(256, 2, strides=2, padding="same")(d1)
    u2 = layers.Concatenate()([u2, att2]); d2 = residual_block(u2, 256)
    u3 = layers.Conv2DTranspose(128, 2, strides=2, padding="same")(d2)
    u3 = layers.Concatenate()([u3, c2]); d3 = residual_block(u3, 128)
    u4 = layers.Conv2DTranspose(64, 2, strides=2, padding="same")(d3)
    u4 = layers.Concatenate()([u4, c1]); d4 = residual_block(u4, 64)
    outputs = layers.Conv2D(1, 1, activation="sigmoid")(d4)
    return models.Model(inputs, outputs, name="Attn-ResU-Net")
def dense_block(x, filters):
    x1 = layers.Conv2D(filters, 3, padding="same", activation="relu")(x)
    x2 = layers.Conv2D(filters, 3, padding="same", activation="relu")(layers.Concatenate()([x, x1]))
    x3 = layers.Conv2D(filters, 3, padding="same", activation="relu")(layers.Concatenate()([x, x1, x2]))
    return layers.Concatenate()([x, x1, x2, x3])
def multi_scale_conv(x, filters):
    conv3 = layers.Conv2D(filters, 3, padding="same", activation="relu")(x)
    conv5 = layers.Conv2D(filters, 5, padding="same", activation="relu")(x)
    return layers.Concatenate()([conv3, conv5])
def build_asdms(input_shape):
    inputs = layers.Input(shape=input_shape)
    e1 = dense_block(inputs, 32); p1 = layers.MaxPooling2D(2)(e1)
    e2 = dense_block(p1, 64); p2 = layers.MaxPooling2D(2)(e2)
    e3 = dense_block(p2, 128); p3 = layers.MaxPooling2D(2)(e3)
    e4 = dense_block(p3, 256); p4 = layers.MaxPooling2D(2)(e4)
    b = multi_scale_conv(p4, 512)
    g1 = layers.Conv2D(256, 1, padding="same")(b)
    att1 = attention_gate(e4, g1, 128)
    u1 = layers.Conv2DTranspose(256, 2, strides=2, padding="same")(b)
    u1 = layers.Concatenate()([u1, att1]); d1 = dense_block(u1, 256)
    u2 = layers.Conv2DTranspose(128, 2, strides=2, padding="same")(d1)
    u2 = layers.Concatenate()([u2, e3]); d2 = dense_block(u2, 128)
    u3 = layers.Conv2DTranspose(64, 2, strides=2, padding="same")(d2)
    u3 = layers.Concatenate()([u3, e2]); d3 = dense_block(u3, 64)
    u4 = layers.Conv2DTranspose(32, 2, strides=2, padding="same")(d3)
    u4 = layers.Concatenate()([u4, e1]); d4 = dense_block(u4, 32)
    outputs = layers.Conv2D(1, 1, activation="sigmoid")(d4)
    return models.Model(inputs, outputs, name="ASDMS-Attn-U-Net")
model_builders = [
    build_unet,
    build_resunet,
    build_attnunet,
    build_attnresunet,
    build_asdms
]
results = {}
for builder in model_builders:
    name = builder.__name__.replace("build_", "")
    print(f"\n Training {name} ...")
    model = builder(input_shape=X_train.shape[1:])
    model.compile(optimizer=optimizers.Adam(LR),
                  loss="binary_crossentropy",
                  metrics=["accuracy", tf.keras.metrics.MeanIoU(num_classes=2)])
    hist = model.fit(
        X_train, y_train,
        validation_split=0.2,
        epochs=EPOCHS,
        batch_size=BATCH_SIZE,
        verbose=1
    )
    test_metrics = model.evaluate(X_test, y_test, verbose=0)
    results[name] = {
        "loss": test_metrics[0],
        "accuracy": test_metrics[1],
        "iou": test_metrics[2]
    }
    model.save(os.path.join(OUT_DIR, f"{name}.keras"))
print("\n📊 Final IoU Comparison on Test Set:")
for name, res in results.items():
    print(f"{name:<20}  IoU={res['iou']:.4f}  Acc={res['accuracy']:.4f}")


Testing

In [ ]:
import os, numpy as np, matplotlib.pyplot as plt, rasterio
from tensorflow.keras.models import load_model
from rasterio.windows import Window
RAW_IMAGE = "/content/drive/MyDrive/Villangad_2020/data/raw/20241110_053942_45_24f7_3B_AnalyticMS_SR_8b_clip.tif"
RAW_MASK  = "/content/drive/MyDrive/Villangad_2020/data/raw/20241110_053942_45_24f7_3B_AnalyticMS_SR_8b_clip_Hybrid_mask.tif"
MODEL_DIR = "/content/drive/MyDrive/Villangad_2020/experiments_all"
SAVE_PATH = "/content/drive/MyDrive/Villangad_2020/experiments_all/full_comparison_scene.png"
with rasterio.open(RAW_IMAGE) as src:
    img = np.moveaxis(src.read(), 0, 2).astype(np.float32)
    img = img / (np.max(img) + 1e-8)
with rasterio.open(RAW_MASK) as msk_src:
    mask = msk_src.read(1).astype(np.uint8)
print(f"Image shape: {img.shape}, Mask shape: {mask.shape}")
def tile_image(img, tile_size=(256, 256)):
    tiles, coords = [], []
    h, w, _ = img.shape
    for top in range(0, h, tile_size[0]):
        for left in range(0, w, tile_size[1]):
            bottom = min(top + tile_size[0], h)
            right  = min(left + tile_size[1], w)
            tile = img[top:bottom, left:right]
            if tile.shape[0] < tile_size[0] or tile.shape[1] < tile_size[1]:
                pad_h = tile_size[0] - tile.shape[0]
                pad_w = tile_size[1] - tile.shape[1]
                tile = np.pad(tile, ((0, pad_h), (0, pad_w), (0,0)), mode='reflect')
            tiles.append(tile)
            coords.append((top, left, bottom, right))
    return np.array(tiles), coords
def merge_tiles(preds, coords, shape, tile_size=(256,256)):
    """Merge predicted tiles (with channel dim) back into full image."""
    full_pred = np.zeros((shape[0], shape[1], 1), dtype=np.float32)
    for pred, (top, left, bottom, right) in zip(preds, coords):
        pred = np.squeeze(pred)  # remove channel dim
        h, w = bottom - top, right - left
        full_pred[top:top+h, left:left+w, 0] = pred[:h, :w]
    return np.squeeze(full_pred)
model_names = [
    "unet",
    "resunet",
    "attnunet",
    "attnresunet",
    "asdms"
]
models_dict = {}
for name in model_names:
    path = os.path.join(MODEL_DIR, f"{name}.keras")
    models_dict[name] = load_model(path, compile=False)
    print(f" Loaded: {name}")
print("🔮 Predicting full scene for all models...")
tiles, coords = tile_image(img)
print(f"Total tiles: {len(tiles)}")
pred_maps = {}
for name, model in models_dict.items():
    preds = model.predict(tiles, verbose=0)
    preds_bin = (preds > 0.5).astype(np.uint8)
    merged = merge_tiles(preds_bin, coords, img.shape)
    pred_maps[name] = merged
    print(f" {name} prediction complete")
rgb = img[..., [3, 2, 1]]
rgb = (rgb - np.min(rgb)) / (np.max(rgb) - np.min(rgb) + 1e-8)
plt.figure(figsize=(25, 6))
titles = ["Satellite RGB", "Ground Truth"] + [name.replace("_", " ").upper() for name in model_names]
images = [rgb, mask] + [pred_maps[n] for n in model_names]
for i, (title, im) in enumerate(zip(titles, images), 1):
    plt.subplot(1, len(images), i)
    if i == 1:
        plt.imshow(im)
    else:
        plt.imshow(im, cmap="gray")
    plt.title(title, fontsize=10)
    plt.axis("off")
plt.tight_layout()
plt.savefig(SAVE_PATH, dpi=300, bbox_inches="tight")
plt.show()
print(f" Saved single composite comparison image at:\n{SAVE_PATH}")